In [1]:
import numpy as np
from keras import layers
from keras.layers import Input,Add,Dense,Activation,ZeroPadding2D,BatchNormalization,Flatten,Conv2D,AveragePooling2D,MaxPooling2D,GlobalMaxPooling2D
from keras.models import Model,load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from ResNet_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)


Using TensorFlow backend.


In [2]:
X_train_orig,Y_train_orig,X_test_orig,Y_test_orig,classes=load_dataset()
X_train=X_train_orig/255
X_test=X_test_orig/255
Y_train=convert_to_one_hot(Y_train_orig,6).T
Y_test=convert_to_one_hot(Y_test_orig,6).T

In [3]:
def identity_block(X,f,filters,stage,block):
    conv_name_base='res'+str(stage)+block+'_branch'
    bn_name_base='bn'+str(stage)+block+'_branch'
    F1,F2,F3=filters
    X_shortcut=X
    #Layer 1
    X=Conv2D(filters= F1,kernel_size=(1,1),strides=(1,1),padding='valid',name=conv_name_base+'2a',kernel_initializer=glorot_uniform())(X)
    X=BatchNormalization(axis=3,name=bn_name_base+'2a')(X)
    X=Activation('relu')(X)
    #layer 2
    X=Conv2D(filters=F2,kernel_size=(f,f),strides=(1,1),padding='same',name=conv_name_base+'2b',kernel_initializer=glorot_uniform())(X)
    X=BatchNormalization(axis=3,name=bn_name_base+'2b')(X)
    X=Activation('relu')(X)
    #layer 3
    X=Conv2D(filters=F3,kernel_size=(1,1),strides=(1,1),padding='valid',name=conv_name_base+'2c',kernel_initializer=glorot_uniform())(X)
    X=BatchNormalization(axis=3,name=bn_name_base+'2c')(X)
    X=Activation('relu')(X)
    #Add value of a[l] before a[l+3]
    X=Add()([X,X_shortcut])
    X=Activation('relu')(X)
    return X

def convolutional_block(X,f,filters,stage,block,s=2):
    conv_name_base='res'+str(stage)+block+'_branch'
    bn_name_base='bn'+str(stage)+block+'_branch'
    F1,F2,F3=filters
    X_shortcut=X
    #layer 1
    X=Conv2D(filters=F1,kernel_size=(1,1),strides=(s,s),padding='valid',name=conv_name_base+'2a',kernel_initializer=glorot_uniform())(X)
    X=BatchNormalization(axis=3,name=bn_name_base+'2a')(X)
    X=Activation('relu')(X)
    #layer 2
    X=Conv2D(filters=F2,kernel_size=(f,f),strides=(1,1),padding='same',name=conv_name_base+'2b',kernel_initializer=glorot_uniform())(X)
    X=BatchNormalization(axis=3,name=bn_name_base+'2b')(X)
    X=Activation('relu')(X)
    #layer 3
    X=Conv2D(filters=F3,kernel_size=(1,1),strides=(1,1),padding='valid',name=conv_name_base+'2c',kernel_initializer=glorot_uniform())(X)
    X=BatchNormalization(axis=3,name=bn_name_base+'2c')(X)
    X=Activation('relu')(X)
    #adding 1 convolutional layer in X_shortcut.Reason:To make it of same size as X
    X_shortcut=Conv2D(filters=F3,kernel_size=(1,1),strides=(s,s),name=conv_name_base+'1',kernel_initializer=glorot_uniform())(X)
    X_shortcut=BatchNormalization(axis=3,name=bn_name_base+'1')(X)
    #Now add
    X=Add()([X,X_shortcut])
    X=Activation('relu')(X)
    return X

def ResNet50(input_shape=(64,64,3),classes=6):
    """"
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER
    """
    #tensor created
    X_input=Input(input_shape)
    X=ZeroPadding2D((3,3))(X_input)
    #stage 1 
    X=Conv2D(filters=64,kernel_size=(7,7),strides=(1,1),name='conv1',kernel_initializer=glorot_uniform())(X)
    X=BatchNormalization(axis=3,name='bn1')(X)
    X=Activation('relu')(X)
    X=MaxPooling2D((3,3),strides=(2,2))(X)
    #stage 2
    X=convolutional_block(X,f=3,filters=[64,64,256],stage=2,block='a',s=1)
    X=identity_block(X,3,[64,64,256],stage=2,block='b')
    X=identity_block(X,3,[64,64,256],stage=2,block='c')
    #stage 3
    X=convolutional_block(X,f=3,filters=[128,128,512],stage=3,block='a',s=2)
    X=identity_block(X,3,[128,128,512],stage=3,block='b')
    X=identity_block(X,3,[128,128,512],stage=3,block='c')
    X=identity_block(X,3,[128,128,512],stage=3,block='d')
    #stage 4
    X=convolutional_block(X,f=3,filters=[256,256,1024],stage=4,block='a',s=2)
    X=identity_block(X,3,[256,256,1024],stage=4,block='b')
    X=identity_block(X,3,[256,256,1024],stage=4,block='c')
    X=identity_block(X,3,[256,256,1024],stage=4,block='d')
    X=identity_block(X,3,[256,256,1024],stage=4,block='e')
    X=identity_block(X,3,[256,256,1024],stage=4,block='f')
    #stage 5
    X=convolutional_block(X,f=3,filters=[512,512,2048],stage=5,block='a',s=2)
    X=identity_block(X,3,[512,512,2048],stage=5,block='b')
    X=identity_block(X,3,[512,512,2048],stage=5,block='c')
    #average pooling
    X=AveragePooling2D(pool_size=(2,2),padding='same')(X)
    #output layer
    X=Flatten()(X)
    X=Dense(classes,activation='softmax',name='fc'+str(classes),kernel_initializer=glorot_uniform())(X)
    model=Model(inputs=X_input,outputs=X,name='resNet50')
    return model


    

In [4]:
#now comile the function
model=ResNet50(input_shape=(64,64,3),classes=6)

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train,Y_train,epochs=8,batch_size=32)

Epoch 1/8
1080/1080 [==============================] - 499s 462ms/step - loss: 5.3386 - accuracy: 0.2889
Epoch 2/8
1080/1080 [==============================] - 423s 391ms/step - loss: 2.0268 - accuracy: 0.5519
Epoch 3/8
1080/1080 [==============================] - 417s 386ms/step - loss: 0.8755 - accuracy: 0.7472
Epoch 4/8
1080/1080 [==============================] - 418s 387ms/step - loss: 0.9001 - accuracy: 0.7111
Epoch 5/8
1080/1080 [==============================] - 416s 385ms/step - loss: 0.4588 - accuracy: 0.8343
Epoch 6/8
1080/1080 [==============================] - 416s 386ms/step - loss: 0.9493 - accuracy: 0.8398
Epoch 7/8
1080/1080 [==============================] - 416s 385ms/step - loss: 0.6272 - accuracy: 0.8500
Epoch 8/8
1080/1080 [==============================] - 421s 390ms/step - loss: 0.2840 - accuracy: 0.9028


In [5]:
preds=model.evaluate(X_test,Y_test)
test_accuracy=print("Test Accuracy"+str(preds[1]))

120/120 [==============================] - 17s 146ms/step
Test Accuracy0.8666666746139526
